In [70]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
import os
import time
import numpy as np
client_id = "72c9f2b9d3344c91af50a24f4144228f"
client_secret = "249943a9bbb74b9a8bf04bb7fd93790d"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) # sp is the instance of the spotipy api


In [82]:
data = pd.read_csv('./LY_Artist_lyrics_genre_data_from_big5_mft_users_likes_final.csv')

data = data[data['lang_detect_spacy']=='en'] # Selecting the data which has the lang_detect_spacy column as "en"
dataset_artists = list(data['Artist']) # Creating a list of all the artists from our dataset
dataset_titles = list(data['title']) # Creating a list of all the titles from our dataset
data

,Artist,title,original_lyrics,hasLyrics,isOrchestral,first_genre_tag,second_genre_tag,cleaned_lyrics,lang_detect_spacy
0,Nsync,Bye Bye Bye,"[Intro: Justin & All]\nHey, hey\nBye bye bye\n...",Yes,No,Pop,No-tag,"Hey, hey Bye bye bye Bye bye! Bye bye! I'm ...",en
1,Nsync,It’s Gonna Be Me,[Intro: Justin]\n(It's gonna be me)\nOooh yeah...,Yes,No,Pop,No-tag,"Oooh yeah You might've been hurt, babe That ...",en
2,Nsync,Tearin’ Up My Heart,[Chorus: JC & Justin]\nIt's tearin' up my hear...,Yes,No,Pop,No-tag,It's tearin' up my heart when I'm with you But...,en
3,Nsync,Gone,[Verse 1: Justin]\nThere's a thousand words th...,Yes,No,Pop,No-tag,There's a thousand words that I could say To m...,en
4,Nsync,"Merry Christmas, Happy Holidays","[Intro: Justin, All & JC]\nOooh, ooh ooh\nMerr...",Yes,No,Pop,No-tag,"Oooh, ooh ooh Merry Christmas Happy holidays M...",en
...,...,...,...,...,...,...,...,...,...
53530,​Twenty One Pilots,Car Radio,[Chorus]\nI ponder of something great\nMy lung...,Yes,No,Pop,Rap/Hip Hop,I ponder of something great My lungs will fill...,en
53531,​Twenty One Pilots,Heavydirtysoul,[Instrumental Intro]\n\n[Verse 1]\nThere's an ...,Yes,No,Pop,Rap/Hip Hop,There's an infestation in my mind's imaginatio...,en
53532,​Twenty One Pilots,Tear in My Heart,[Intro]\nAn-nyŏng-ha-se-yo\n\n[Verse 1]\nSomet...,Yes,No,Pop,Rap/Hip Hop,An-nyŏng-ha-se-yo Sometimes you gotta bleed ...,en
53533,​Twenty One Pilots,My Blood,[Verse 1]\nWhen everyone you thought you knew\...,Yes,No,Pop,Rap/Hip Hop,When everyone you thought you knew Deserts you...,en


In [83]:
'''
Creating a list which has both artists and their songs
'''
            
artist_title_dataset = []
for i in range(len(data)):
    artist_title_dataset.append(str(data['Artist'].iloc[i]) + ' ' + str(data['title'].iloc[i]))
artist_title_dataset

['Nsync Bye Bye Bye',
 'Nsync It’s Gonna Be Me',
 'Nsync Tearin’ Up My Heart',
 'Nsync Gone',
 'Nsync Merry Christmas, Happy Holidays',
 'Nsync It Makes Me Ill',
 'Nsync This I Promise You',
 'Nsync Pop',
 'Nsync I Want You Back',
 'Shels Butterflies on Luci’s Way',
 'Shels Journey to the Plains',
 'Shels Plains of the Purple Buffalo Pt. 2',
 'Shels The conference of the birds',
 'Shels Conqueror',
 'Shels The Spirit Horse',
 'Shels Vision Quest',
 'Shels Atoll',
 '1 Giant Leap My Culture',
 '1 Giant Leap Braided Hair',
 '1 Giant Leap Passion',
 '1 Giant Leap Daphne',
 '1 Giant Leap Arrival',
 '1 Giant Leap The Way You Dream',
 '1 Giant Leap All Alone (On Eilean Shona)',
 '1 Giant Leap Racing Away',
 '1 Giant Leap Ma’ Africa',
 '1000Mods Vidage',
 '1000Mods She',
 '1000Mods Electric Carve',
 '1000Mods Low',
 '1000Mods Road to Burn',
 '1000Mods Claws',
 '1000Mods Loose',
 '1000Mods Super Van Vacation',
 '1000Mods The Son',
 '1000Mods On a Stone',
 '10Cc I’m Not in Love',
 '10Cc Dreadloc

In [84]:
records_ascii = []
records_foreign_char= []

def is_ascii(s):
  return s.isascii()

for item in artist_title_dataset:
    if is_ascii(item) is True:
        records_ascii.append(item)
    else:
        records_foreign_char.append(item)

In [85]:
records_ascii[0]

'Nsync Bye Bye Bye'

In [86]:
print('The total number of records with only ascii :- ',len(records_ascii))
print('Whereas, the total number of records with foreign characters are :- ',len(records_foreign_char))

The total number of records with only ascii :-  40598
Whereas, the total number of records with foreign characters are :-  5640


<h3>In the below cell I will remove all the non ascii characters from the 8546 records and query the resulting string to spotify's api. Spotify will understand the name of the artist even if there are one or two characters misplaced or missing. Hence we will use this to clean our artist names. We will replace our faulty artist names with the artist anmes that spotify gives when we make a query using the records without the foreign characters.

The below regular expression will remove all the non ascii characters from our string

In [87]:
import re

def only_ascii(item):
    rex = re.sub(r'[^\x00-\x7F]+',' ', item)
    return rex

In [88]:
artist_ascii = []
title_ascii = []
for artist in dataset_artists:
     artist_ascii.append(only_ascii(str(artist)))

for title in dataset_titles:
     title_ascii.append(only_ascii(str(title)))


In [51]:
''' Creating a combined list of artists and titles'''
ascii_artist_title = []
for item in range(len(artist_ascii)):
    ascii_artist_title.append(artist_ascii[item] + ' ' + title_ascii[item])

In [89]:
'''
In track_preview_available function I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available.

In is_same_artist function I check whether the artist the we found in our query is the same one from our dataset.
'''
def is_same_artist(query, data, index):
    if query['tracks']['items'][0]['artists'][0]['name'] in data[index]:
        return True
def track_preview_available(query):
    if query['tracks']['items']:
        if query['tracks']['items'][0]['preview_url']:
            return True

<h3> Correcting artist names

In [93]:
len(data) == len(artist_ascii)

True

In [100]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()
artist_from_spotify = []
artist_names_changed = []

for index in range(len(artist_ascii[:50])):
    artist_name = artist_ascii[index]
    song_title = title_ascii[index]
    search = f'artist:{artist_name} track:{song_title}'
    query = sp.search(search, type='track')
    
    if track_preview_available(query) and is_same_artist(query,ascii_artist_title,index):
        artist_from_spotify.append(query['tracks']['items'][0]['artists'][0]['name'])
        artist_names_changed.append(query['tracks']['items'][0]['artists'][0]['name'])

    else:
        artist_from_spotify.append(artist_ascii[index])

    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 artists uris fetched
Loop #: 5
Elapsed Time: 3.2292048931121826 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 6.389528036117554 seconds
15 artists uris fetched
Loop #: 15
Elapsed Time: 9.018861055374146 seconds
20 artists uris fetched
Loop #: 20
Elapsed Time: 11.82250690460205 seconds
25 artists uris fetched
Loop #: 25
Elapsed Time: 15.273070096969604 seconds
30 artists uris fetched
Loop #: 30
Elapsed Time: 18.808233976364136 seconds
35 artists uris fetched
Loop #: 35
Elapsed Time: 21.877492904663086 seconds
40 artists uris fetched
Loop #: 40
Elapsed Time: 25.117040872573853 seconds
45 artists uris fetched
Loop #: 45
Elapsed Time: 29.775147914886475 seconds
50 artists uris fetched
Loop #: 50
Elapsed Time: 33.20346474647522 seconds


<h3> Now we have cleaned artist names, and hence we can now use these names and extract the sings preview_urls, track_ids, artist_ids, popularity and other faetures

In [98]:
artist_from_spotify

['*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*NSYNC',
 '*shels',
 '*shels',
 '*shels Plains of the Purple Buffalo Pt. 2',
 '*shels',
 '*shels',
 '*shels',
 '*shels',
 '*shels',
 '*shels Vision Quest',
 '*shels Atoll',
 '1 Giant Leap My Culture',
 '1 Giant Leap Braided Hair',
 '1 Giant Leap Dunya Salam',
 '1 Giant Leap Passion',
 '1 Giant Leap Daphne',
 '1 Giant Leap Arrival',
 '1 Giant Leap The Way You Dream',
 '1 Giant Leap All Alone (On Eilean Shona)',
 '1 Giant Leap Racing Away',
 '1 Giant Leap Ma  Africa',
 '1000mods',
 '1000mods',
 '1000mods',
 '1000mods',
 '1000mods',
 '1000mods',
 '1000mods',
 '1000mods Super Van Vacation',
 '1000mods The Son',
 '1000mods On a Stone',
 '10cc',
 '10cc Dreadlock Holiday',
 '10cc The Things We Do for Love',
 '10cc Rubber Bullets',
 '10cc I m Mandy, Fly Me',
 '10cc Life Is a Minestrone',
 '10cc Une nuit a Paris',
 '10cc The Wall Street Shuffle',
 '10cc The Worst Band in the World',
 '10cc Good Morning Jud

In [71]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()

artist_from_spotify = []

for index in range(len(artist_ascii[:50])):
    artist_name = artist_ascii[index]
    song_title = title_ascii[index]
    search = f'artist:{artist_name} track:{song_title}'
    query = sp.search(search, type='track')


    # in the below line of code I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available, and
    # even further I check a third condition whether the artist the we found in our query is the same from our dataset.


    # if query['tracks']['items'] and query['tracks']['items'][0]['preview_url'] and query['tracks']['items'][0]['artists'][0]['name'] in alpha_artists_titles[index]:
    if track_preview_available(query) and is_same_artist(query,ascii_artist_title,index):
        artist_from_spotify.append(query['tracks']['items'][0]['artists'][0]['name'])

    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 artists uris fetched
Loop #: 5
Elapsed Time: 1.8791790008544922 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 5.853464126586914 seconds
15 artists uris fetched
Loop #: 15
Elapsed Time: 9.114161252975464 seconds
20 artists uris fetched
Loop #: 20
Elapsed Time: 11.25255036354065 seconds
25 artists uris fetched
Loop #: 25
Elapsed Time: 13.402014970779419 seconds
30 artists uris fetched
Loop #: 30
Elapsed Time: 16.36712908744812 seconds
35 artists uris fetched
Loop #: 35
Elapsed Time: 18.885977268218994 seconds
40 artists uris fetched
Loop #: 40
Elapsed Time: 22.002456188201904 seconds
45 artists uris fetched
Loop #: 45
Elapsed Time: 25.801711320877075 seconds
50 artists uris fetched
Loop #: 50
Elapsed Time: 27.914245128631592 seconds


29